In [2]:
import os
import pandas as pd

import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize

import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
import keras.callbacks as kcall
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Convolution2D,ZeroPadding2D, MaxPooling2D, BatchNormalization
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense
from keras.optimizers import SGD

%matplotlib inline

/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: Future

In [3]:
train_dir = 'data/train-only-sharpen'

In [4]:
for root,dirs,files in os.walk(train_dir):
    print (root, len(files))

data/train-preprocess 0
data/train-preprocess/MEL 1113
data/train-preprocess/DF 115
data/train-preprocess/BKL 1099
data/train-preprocess/BCC 514
data/train-preprocess/VASC 142
data/train-preprocess/AKIEC 327
data/train-preprocess/NV 6705


In [5]:
from keras.preprocessing.image import ImageDataGenerator

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest',
    horizontal_flip=True,
    validation_split=0.25)

# target_size = (height, width)
target_size = (450, 600)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = target_size,       
        class_mode = 'categorical',
        batch_size=32,
        subset="training",
        shuffle = True)

validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = target_size,        
        class_mode = 'categorical',
        batch_size=32,
        subset = "validation",
        shuffle = True)

Found 7515 images belonging to 7 classes.
Found 2500 images belonging to 7 classes.


In [6]:
output_classes = 7

batch_size = 32 
epochs = 30
learning_rate= 0.001
# input_shape=(img_height, img_width, img_ckannels),
input_shape = (450,600,3)

optimizer = 'adam'

In [7]:
def get_vgg(input_shape, output_classes):
    vgg = Sequential()
    vgg.add(ZeroPadding2D((1,1),input_shape = input_shape))
    vgg.add(Convolution2D(64, 3, 3, activation='relu'))
    vgg.add(ZeroPadding2D((1,1)))
    vgg.add(Convolution2D(64, 3, 3, activation='relu'))
    vgg.add(MaxPooling2D((2,2), strides=(2,2)))
    
    vgg.add(ZeroPadding2D((1,1)))
    vgg.add(Convolution2D(128, 3, 3, activation='relu'))
    vgg.add(ZeroPadding2D((1,1)))
    vgg.add(Convolution2D(128, 3, 3, activation='relu'))
    vgg.add(MaxPooling2D((2,2), strides=(2,2)))
    
    vgg.add(ZeroPadding2D((1,1)))
    vgg.add(Convolution2D(256, 3, 3, activation='relu'))
    vgg.add(ZeroPadding2D((1,1)))
    vgg.add(Convolution2D(256, 3, 3, activation='relu'))
    vgg.add(ZeroPadding2D((1,1)))
    vgg.add(Convolution2D(256, 3, 3, activation='relu'))
    vgg.add(MaxPooling2D((2,2), strides=(2,2)))
    
    vgg.add(ZeroPadding2D((1,1)))
    vgg.add(Convolution2D(512, 3, 3, activation='relu'))
    vgg.add(ZeroPadding2D((1,1)))
    vgg.add(Convolution2D(512, 3, 3, activation='relu'))
    vgg.add(ZeroPadding2D((1,1)))
    vgg.add(Convolution2D(512, 3, 3, activation='relu'))
    vgg.add(MaxPooling2D((2,2), strides=(2,2)))
    
    vgg.add(ZeroPadding2D((1,1)))
    vgg.add(Convolution2D(512, 3, 3, activation='relu'))
    vgg.add(ZeroPadding2D((1,1)))
    vgg.add(Convolution2D(512, 3, 3, activation='relu'))
    vgg.add(ZeroPadding2D((1,1)))
    vgg.add(Convolution2D(512, 3, 3, activation='relu'))
    vgg.add(MaxPooling2D((2,2), strides=(2,2)))
    
    vgg.add(Flatten())
    vgg.add(Dense(4096, activation='relu'))
    vgg.add(Dropout(0.5))
    vgg.add(Dense(4096, activation='relu'))
    vgg.add(Dropout(0.5))
    vgg.add(Dense(output_classes, activation='softmax'))
    return vgg

In [8]:
vgg = get_vgg(input_shape,output_classes)
vgg.summary()

/home/sara/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
/home/sara/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
/home/sara/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
/home/sara/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  if sys.path[0] == '':
/home/sara/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  app.la

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 452, 602, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 450, 600, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 452, 602, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 450, 600, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 225, 300, 64)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 227, 302, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 225, 300, 128)     73856     
__________

In [9]:
vgg.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
history = vgg.fit_generator(
      train_generator,
      batch_size = batch_size,
      steps_per_epoch = 400,
      epochs = epochs,
      validation_data = validation_generator,
      validation_steps = 100)

TypeError: fit_generator() got an unexpected keyword argument 'batch_size'

In [ ]:
score = vgg.evaluate_generator(validation_generator, steps=50)

print ('Test Score: ', score[0])
print ('Test Accuracy: ',score[1])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
filename = validation_generator.filenames
truth = validation_generator.classes
label = validation_generator.class_indices
indexlabel = dict((value, key) for key, value in label.items())

In [ ]:
predicts = vgg.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size, verbose=1)
predict_class = np.argmax(predicts, axis=1)
errors = np.where(predict_class != truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(truth,predict_class)

labels = []
for k,v in indexlabel.items():
    labels.append(v)
    
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, classes=labels,
                      title='Confusion matrix, without normalization')

In [ ]:
def predict_and_report(gen, model):
    y_true = []
    y_pred = []
    gen.reset()
    for img, label in gen:
        #get true labels for batch and store them
        y_true.extend([int(z[1]) for z in label])
        #Get predictions as probabilities
        batch_pred = model.predict_on_batch(img)
        #turn probabilities to class labels and store
        batch_pred = np.argmax(batch_pred, axis=1)
        y_pred.extend(batch_pred)
        #break loop
        if gen.batch_index == 0:
            break
            
    print('Accuracy:', accuracy_score(y_true, y_pred))
    print('Area Under the Receiver Operating Characteristic Curve:', roc_auc_score(y_true, y_pred)) #Area under the curve
    print(classification_report(y_true, y_pred))

In [ ]:
predict_and_report(train_generator, vgg)
predict_and_report(validation_generator, vgg)

In [ ]:
best_train_acc = max(history.history['acc'])
best_train_acc

In [ ]:
last_train_acc = history.history['acc'][-1]
last_train_acc

In [ ]:
results.append([train_dir, optimizer,learning_rate, batch_size, epochs, last_train_acc, best_train_acc, score[0], score[1]])

In [ ]:
import  pandas  as  pd
res = pd.DataFrame(results)
res.columns = ['data', 'optimizer','learning_rate','batch_size', 'epochs','last_train_accuracy', 'best_training_accuracy','test Score', 'test_accuracy']
res

In [ ]:
res.to_csv("VGGNet2-With.csv", sep=',', encoding='utf-8')